In [ ]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000 to get all the rows for our dataframe
# pd.set_option('display.max_colwidth', None)  # or 199


In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")

In [ ]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
train_df.head(10)

So, there are some 119 features and and a target variable claim. 
Let's first try to train the model without null values and will then impute the null values with some value. 

In [ ]:
Summary = pd.DataFrame(train_df.dtypes, columns=['Dtype'])
Summary["max"] = train_df.max()
Summary["min"] = train_df.min()
Summary["Null"] = train_df.isnull().sum() # to get null values
Summary["First"] = train_df.iloc[0] # to get first value
Summary["Second"] = train_df.iloc[1] # to get second value
Summary


Almost all the rows have null values and the datatypes for all column is float except for claim, that is target variable, in that case it is int.

Let's drop the target variable and id from the training data.

In [ ]:
# train_df = train_df.fillna(0)
train_df = train_df.fillna(train_df.median())

In [ ]:
print(len(train_df))

In [ ]:
test_df.head(10)

We can see that the test_df also have 119 features. Target variable is not included in the dataset. 

Let's drop the id column from the test dataset as well and impute the missing values with zero.

In [ ]:
# y.nunique.count()

In [ ]:
test_df = test_df.drop("id", axis=1)

In [ ]:
test_df = test_df.fillna(test_df.median())


In [ ]:
y = train_df["claim"]
train_df = train_df.drop(["id"], axis=1)

In [ ]:
import matplotlib.pyplot as plt
corr_mat = train_df.corr()

In [ ]:
# corr_mat[abs(corr_mat) > 0.5]

cor_target = abs(corr_mat["claim"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target<0.0009]
relevant_features

In [ ]:
feat_to_drop = relevant_features.index.to_list()
train_df = train_df.drop(feat_to_drop, axis = 1)
test_df = test_df.drop(feat_to_drop, axis = 1)

In [ ]:
train_df = train_df.drop(["claim"], axis=1)

In [ ]:
# test_df.isnull().sum()
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(train_df, y,random_state = 123, test_size = 0.1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from xgboost import XGBRegressor
# import lightgbm

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

In [ ]:
lr = LinearRegression()
# lr = LogisticRegression(random_state=123)
lr.fit(X_train, y_train)


In [ ]:
xgbr= XGBRegressor(
    n_estimators=3000,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.1,
    max_depth=3,
    booster='gbtree',
    reg_lambda=0.0008,
    reg_alpha=24,
    random_state=123
)
# xgbr = XGBRegressor(random_state = 123)
xgbr.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgbr.predict(X_test)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
y_pred

In [ ]:
# y_pred_reshaped = []
# for val in y_pred:
#     if val[0] > val[1]:
#         y_pred_reshaped.append()

In [ ]:
y_pred.shape

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(auc(fpr, tpr))
# this is the score: 0.5254161343242256
# this is the score with logisticR: 0.530162419244593

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, y_pred_xgb)
print(auc(fpr, tpr))
# this is the score: 0.776245437662909

In [ ]:
test_df = scaler.transform(test_df)

In [ ]:
y_val = lr.predict(test_df)

In [ ]:
y_val_xgbr = xgbr.predict(test_df)

In [ ]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")

In [ ]:
y_val

In [ ]:
y_val_xgbr

In [ ]:
sub["claim"] = y_val_xgbr

In [ ]:
y_val_xgbr

In [ ]:
y_res = (y_val_xgbr + y_val)/2

In [ ]:
sub["claim"] = y_val_xgbr
sub.to_csv("submission.csv", index=False)
sub.head(10)